In [133]:
import psycopg2


SERVER_IP = '192.168.122.153'
tables = {}  # {название_таблицы: {'coulumns': [(название_колонок, тип_данных)], 'records_count': кол-во строк}. 

conn_string_from = f"host='{SERVER_IP}' port=54320 dbname='my_database' user='root' password='postgres'" 
conn_string_to = f"host='{SERVER_IP}' port=5433 dbname='my_database' user='root' password='postgres'" 

# получим все названия таблиц, чтобы скопировать из них информаицю
with psycopg2.connect(conn_string_from) as conn, conn.cursor() as cursor:
    query = 'SELECT relname, n_live_tup FROM pg_stat_user_tables ORDER BY n_live_tup DESC;'
    cursor.execute(query)
    tbls = cursor.fetchall()

    # Запомним названия и типы колонок из каждой таблицы, чтобы создать в новой базе.
    # Не будем сохранять дополнительные атрибуты типа ключей и NOT NULL
    for table_name, rows_count in tbls:
        query = f"SELECT \
                        a.attname as Column, \
                        pg_catalog.format_type(a.atttypid, a.atttypmod) as Datatype \
                    FROM \
                        pg_catalog.pg_attribute a \
                   WHERE \
                        a.attnum > 0 \
                        AND NOT a.attisdropped \
                        AND a.attrelid = ( \
                            SELECT c.oid \
                            FROM pg_catalog.pg_class c \
                                LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace \
                            WHERE c.relname ~ '^({table_name})$' \
                                AND pg_catalog.pg_table_is_visible(c.oid) \
                        );"
        cursor.execute(query)
        clmns = cursor.fetchall()
        
        tables[table_name] = {'coulumns': clmns, 'records_count': rows_count}

In [109]:
# выгружаем данные из старой базы в файлы
conn_string= f"host='{SERVER_IP}' port=54320 dbname='my_database' user='root' password='postgres'" 
with psycopg2.connect(conn_string_from) as conn, conn.cursor() as cursor:
    for table in tables:
        q = f"COPY {table} TO STDOUT WITH DELIMITER ',' CSV HEADER;"
        with open(f'{table}.csv', 'w') as f:
            cursor.copy_expert(q, f)
            print(f'Таблица {table} сохранена на диск')

'Таблица lineitem сохранена на диск'
'Таблица orders сохранена на диск'
'Таблица partsupp сохранена на диск'
'Таблица part сохранена на диск'
'Таблица customer сохранена на диск'
'Таблица supplier сохранена на диск'
'Таблица nation сохранена на диск'
'Таблица region сохранена на диск'


In [110]:
# загружаем данные из файлов в новую базу
with psycopg2.connect(conn_string_to) as conn, conn.cursor() as cursor:
    
    # сделаем таблицы в новой базе как в старой
    for table, data in tables.items():
        # сначала удалим таблицы с подобными именами, которые уже могут там быть
        cursor.execute(f"DROP TABLE IF EXISTS {table}")

        # cоздадим таблицы с идентичными полями
        fields = []
        for name, typ in data['coulumns']: # тут можно изменить типы колонок
            # if 'character' in typ:
                # typ = 'varchar(1)'
            
            fields.append(f'{name} {typ}')

        fileds = ', '.join(fields)
        query = f"CREATE TABLE {table} ({fileds})"
        cursor.execute(query) # выполнение запроса
    
        conn.commit() # комитим транзакцию

    # выполним копирование данных
    for table, _ in tables.items():
        with open(f'{table}.csv', 'r') as f:
            q = f"COPY {table} from STDIN WITH DELIMITER ',' CSV HEADER;"
            cursor.copy_expert(q, f)
            print(f'В новую базу загружен файл {table}')

        conn.commit() # комитим транзакцию

'В новую базу загружен файл lineitem'
'В новую базу загружен файл orders'
'В новую базу загружен файл partsupp'
'В новую базу загружен файл part'
'В новую базу загружен файл customer'
'В новую базу загружен файл supplier'
'В новую базу загружен файл nation'
'В новую базу загружен файл region'


In [137]:
# проверяем как данные сохранились из одной базы в другую, подсчитывая количество строк до и после переноса
with psycopg2.connect(conn_string_to) as conn, conn.cursor() as cursor:
    cursor.execute(f'SELECT relname, n_live_tup FROM pg_stat_user_tables ORDER BY n_live_tup DESC;')
    tables_new = {name: rows_cnt for name, rows_cnt in cursor.fetchall()}
    tables_old = {name: data['records_count'] for name, data in tables.items()}
    # сравниваем со старыми значениями
    bad_items = [k for k in tables_old if k not in tables_new or tables_old[k] != tables_new[k]]
    if len(bad_items):
        for tbl in bad_items:
            print(f'В таблице {tbl} должно быть {tables_old[tbl]} записей, а перенеслось только {tables_new[tbl]}')
            print(f'Исходная таблица содержит следующие колонки:')
            for name, typ in tables[tbl]["coulumns"]:
                print(f'{name} {typ}')
    else:
        print(f'Все данные успешно перенесены')

'В таблице lineitem должно быть 6000925 записей, а перенеслось только 6000505'
'Исходная таблица содержит следующие колонки:'
'l_orderkey integer'
'l_partkey integer'
'l_suppkey integer'
'l_linenumber integer'
'l_quantity numeric(15,2)'
'l_extendedprice numeric(15,2)'
'l_discount numeric(15,2)'
'l_tax numeric(15,2)'
'l_returnflag character(1)'
'l_linestatus character(1)'
'l_shipdate date'
'l_commitdate date'
'l_receiptdate date'
'l_shipinstruct character(25)'
'l_shipmode character(10)'
'l_comment character varying(44)'
